In [1]:
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import string
from pyvi import ViTokenizer, ViPosTagger
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models, callbacks, preprocessing, losses, optimizers, utils

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/My Drive/Colab Notebooks/Text_Generator

/content/drive/My Drive/Colab Notebooks/Text_Generator


In [5]:
%ls

data/                  scrape_news.ipynb           temp/                      word_embedding.ipynb
data_processing.ipynb  search_generate_text.ipynb  text-classification.ipynb
model/                 src/                        text_generator.ipynb


In [6]:
vnexpress = pd.read_csv('./data/vnexpress/csv/vnexpress.csv').drop(columns=['Unnamed: 0'], axis=1)
vnexpress.shape

(1347, 6)

In [7]:
vnexpress.head()

,article_id,content,topic,sub-topic,title,description
0,0,Người mẹ 42 tuổi ở Đà Nẵng vội vã tra hỏi và v...,doi-song,to-am,Cha mẹ học về giáo dục giới tính để dạy con,Nghe con trai 14 tuổi nói đã làm cho chị hàng ...
1,1,"Vợ chồng người phụ nữ ở Mỹ Đức, Hà Nội luôn tự...",doi-song,to-am,Nỗi lòng của phụ huynh có con mang thai tuổi đ...,Khi con gái đang là sinh viên năm nhất một trư...
2,2,"Kati Morton, nhà trị liệu hôn nhân và gia đình...",doi-song,to-am,Giải mã 'Hội chứng con gái đầu lòng',"""Hội chứng con gái đầu lòng"" cho rằng con gái ..."
3,3,"Anh Thắng, chủ tịch Hội làm vườn tỉnh Nghệ An ...",doi-song,to-am,Vườn sân thượng như khu rừng giải nhiệt mùa hè,Nghệ AnMuốn hạ nhiệt cho ngôi nhà khi nắng nón...
4,4,"Vừa kết thúc môn tiếng Anh, Phương bị sốt cao,...",doi-song,to-am,Nghị lực của cô bé ung thư,"Hà NamMột tuần trước, cô bé 14 tuổi từ Bệnh vi..."


In [8]:
vnexpress['tag'] = vnexpress['content'] + vnexpress['title'] + vnexpress['description']
vnexpress.drop(columns=['content', 'title', 'description'], inplace=True)

In [9]:
mask = vnexpress['tag'].isna()
vnexpress[mask]

,article_id,topic,sub-topic,tag
93,93,doi-song,bai-hoc-song,NaN
119,119,doi-song,bai-hoc-song,NaN
214,214,doi-song,nha,NaN
324,324,doi-song,tieu-dung,NaN
357,357,doi-song,tieu-dung,NaN
504,504,giai-tri,phim,NaN
1163,1163,thoi-su,lao-dong-viec-lam,NaN
1179,1179,thoi-su,dan-sinh,NaN
1183,1183,thoi-su,dan-sinh,NaN
1192,1192,thoi-su,dan-sinh,NaN


In [10]:
vnexpress.dropna(axis=0, inplace=True)
vnexpress.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1337 entries, 0 to 1346
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article_id  1337 non-null   int64 
 1   topic       1337 non-null   object
 2   sub-topic   1337 non-null   object
 3   tag         1337 non-null   object
dtypes: int64(1), object(3)
memory usage: 52.2+ KB


In [11]:
vnexpress['topic'].value_counts()

topic
giai-tri    616
doi-song    360
thoi-su     350
khoa-hoc     11
Name: count, dtype: int64

In [12]:
news = vnexpress.copy()
label_encoder = LabelEncoder()
news['topic'] = label_encoder.fit_transform(vnexpress['topic'])

### Khởi tạo dữ liệu đầu vào

In [13]:
def clean_document(doc):
    doc = ViTokenizer.tokenize(doc)
    doc = doc.lower() # lower
    tokens = doc.split() # split into words
    table = str.maketrans('', '', string.punctuation.replace("_", "")) # remove all punctuations
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word]
    return tokens

In [14]:
papers = news['tag'].tolist()
print(len(papers))
papers[0:1]

1337


['Người mẹ 42 tuổi ở Đà Nẵng vội vã tra hỏi và vỡ lẽ hóa ra cậu bé hiểu lầm. Ngọc Trung, con trai chị bắt đầu vào tuổi dậy thì, từng vài lần xuất hiện tình trạng cương cứng, phải xả vào bồn cầu. Một lần chị gái hàng xóm sang dùng nhờ nhà vệ sinh. Hai tháng sau, người này khoe với chị Hằng là đã có thai. Ngọc Trung hốt hoảng lên Internet tìm hiểu, thấy nói "xuất ngoài vẫn có thể mang thai". Cậu bé đinh ninh mình là bố của đứa bé trong bụng chị hàng xóm nên thú tội với mẹ. Thở phào vì câu chuyện không nghiêm trọng như mình tưởng nhưng ngay sau đó chị Bích Hằng nhận ra con đang thiếu hụt kiến thức về giới tính và tình dục trầm trọng. Trong lúc giải thích cho Trung, chị cũng bối rối chưa biết cách diễn đạt cho đúng, đôi khi thấy ngượng ngùng. Tìm các lớp để con có hiểu biết hơn nhưng đều phải đợi lịch, chị rủ thêm 5 người bạn khác, cũng đang có con ở tuổi dậy thì cùng đến quán cà phê, mời một chuyên gia hướng dẫn trong hai buổi. Chị Trần Thị Vân Anh, 50 tuổi, có con đang học lớp 8 tại một 

In [15]:
sequences = []
for seq in tqdm(papers):
  tokens = clean_document(seq)
  line = ' '.join(tokens)
  sequences.append(line)

100%|██████████| 1337/1337 [00:25<00:00, 51.76it/s]


In [16]:
print('Length of sequences:', len(sequences))
sequences[2]

Length of sequences: 1337


'kati_morton nhà trị_liệu hôn_nhân và gia_đình ở santa_monica california mỹ gần đây trong một video tiktok với hơn 6 triệu lượt xem đã liệt_kê những dấu_hiệu của hội_chứng con_gái đầu_lòng gồm cảm_giác mãnh_liệt về trách_nhiệm gia_đình xu_hướng làm hài_lòng mọi người và dễ nổi_giận với những người em ruột cách đây hơn một năm trên mạng xã_hội x từng có một câu hỏi được nhiều người quan_tâm bạn có hạnh_phúc khi là con đầu và là con_gái rất nhiều người bình_luận dưới bài viết rằng họ thường_xuyên tự nhận thấy phải gánh_vác trách_nhiệm trong gia_đình cũng như biết cách nhường_nhịn để làm hài_lòng người khác trong số các anh_chị_em chị cả cũng có xu_hướng phải đối_mặt với những kỳ_vọng cao hơn về thành_tích hành_vi và trách_nhiệm mà cha_mẹ đặt ra thậm_chí họ cũng được giao những công_việc trong gia_đình mà các em nhỏ không bao_giờ được giao điều này cũng đặt con_gái đầu_vào vị_trí chuyên đóng vai kẻ xấu trong nhà khi bị các em mô_tả là hách_dịch hoặc độc_đoán không_những thế họ còn phải đố

In [17]:
# filter several punctuations in 50803 sentences
tokenizer = preprocessing.text.Tokenizer(filters='!“"”#$%&()*+,-./:;<=>?@[\]^`{|}~ ', oov_token='<oov>')
# create vocabulary (corpus) for each word in 50803 sentences
tokenizer.fit_on_texts(sequences)

print(len(tokenizer.word_index))
tokenizer.word_index

28858


{'<oov>': 1,
 'và': 2,
 'của': 3,
 'trong': 4,
 'được': 5,
 'là': 6,
 'cho': 7,
 'có': 8,
 'các': 9,
 'người': 10,
 'với': 11,
 'năm': 12,
 'một': 13,
 'không': 14,
 'để': 15,
 'nhiều': 16,
 'khi': 17,
 'ở': 18,
 'những': 19,
 'từ': 20,
 'đến': 21,
 'về': 22,
 'hơn': 23,
 'theo': 24,
 'nhà': 25,
 'trên': 26,
 'ông': 27,
 'cô': 28,
 'vào': 29,
 'như': 30,
 'làm': 31,
 'đó': 32,
 'cũng': 33,
 'anh': 34,
 'tại': 35,
 'tôi': 36,
 'đã': 37,
 'sau': 38,
 'này': 39,
 'ngày': 40,
 'hai': 41,
 'nói': 42,
 'ra': 43,
 'biết': 44,
 'sẽ': 45,
 'còn': 46,
 'tuổi': 47,
 'đi': 48,
 'lại': 49,
 'con': 50,
 'phải': 51,
 'nhưng': 52,
 'có_thể': 53,
 'tháng': 54,
 'qua': 55,
 'cùng': 56,
 'mới': 57,
 'nên': 58,
 'trước': 59,
 'sự': 60,
 'việc': 61,
 'đồng': 62,
 'bạn': 63,
 'nhất': 64,
 'nước': 65,
 'chỉ': 66,
 'mỗi': 67,
 'cao': 68,
 'bà': 69,
 'phim': 70,
 'bị': 71,
 'việt_nam': 72,
 '4': 73,
 'giúp': 74,
 'vì': 75,
 'lên': 76,
 'lần': 77,
 'mình': 78,
 'lao_động': 79,
 'do': 80,
 'từng': 81,
 'đây': 82

In [18]:
# max_length = news['tag'].map(len).max()
max_length = max([len(i) for i in sequences])
max_length

15867

In [19]:
topic_size = len(news['topic'].unique()) + 1
topic_size

5

In [20]:
sequence_digit = tokenizer.texts_to_sequences(sequences)
X = np.array(preprocessing.sequence.pad_sequences(sequence_digit, maxlen=max_length, padding='pre'))

print(len(X[0]))
print(X[0:2])

15867
[[    0     0     0 ...  7546  1038 17870]
 [    0     0     0 ...   107    40  1107]]


In [21]:
## Sửa định dạng y
y = np.array([news['topic']])
y = y.reshape(-1, 1)
y = utils.to_categorical(y, num_classes=topic_size)

print(y[0:5])

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


### Xây dựng mô hình LSTM

In [22]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

28859

In [23]:
model = models.Sequential([
  layers.Embedding(vocab_size, 128, input_length=max_length),
  layers.BatchNormalization(),
  layers.LSTM(64, return_sequences=True),
  layers.LSTM(64),
  layers.Dropout(0.2),
  layers.Dense(9, activation='relu'),
  layers.Dense(9, activation='relu'),
  layers.Dense(9, activation='relu'),
  layers.Dense(9, activation='relu'),
  layers.Dense(5, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15867, 128)        3693952   
                                                                 
 batch_normalization (Batch  (None, 15867, 128)        512       
 Normalization)                                                  
                                                                 
 lstm (LSTM)                 (None, 15867, 64)         49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 9)                 585       
                                                        

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=25, batch_size=128)

Epoch 1/25


### Dự đoán mẫu

In [ ]:
sample_text = '''
Loạt hình Messi ở buổi lễ được nhiều người chia sẻ trên mạng xã hội, thu hút bình luận, khán giả viết: "Cậu ấy từng đoạt giải ở Cannes, thành ngôi sao thảm đỏ rồi", "Biểu hiện của Messi ở nơi đông người thật từ tốn", "Diễn xuất của cậu ấy rất ấn tượng".

Lần đầu Messi dự thảm đỏ khai mạc LHP Cannes, năm ngoái, chú chó tham gia Anatomy of a Fall, phim của đạo diễn Justine Triet, nói về người phụ nữ bị cáo buộc tội giết chồng, nhân chứng duy nhất của cô là cậu con trai mù và chú chó của họ. Tác phẩm thắng giải cao nhất - Cành Cọ Vàng trong khi Messi giành Palm Dog, giải phụ do các nhà phê bình điện ảnh quốc tế trao tặng cho các con chó, hoặc đàn chó có màn thể hiện tốt. Phần thưởng gồm một chiếc vòng cổ bằng da cho chó có chữ "Palm Dog". Tên hạng mục này là một cách chơi chữ so với giải Cành Cọ Vàng (Palme d'Or).
'''

In [ ]:
tokeninzed_sequences = clean_document(sample_text)
sequences_digit = tokenizer.texts_to_sequences([tokeninzed_sequences])

X_test = np.array(preprocessing.sequence.pad_sequences(sequences_digit, maxlen=max_length, padding='pre'))
print(X_test)

predictions = model.predict(X_test)
print(predictions)

In [ ]:
result = label_encoder.inverse_transform([np.argmax(predictions[0])])
result

### Dump model

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/Text_Generator/model

In [ ]:
import pickle
model.save('text_classify_model.h5')

with open('text_classify_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

with open('text_classify_sequences_digit.pkl', 'wb') as f:
    pickle.dump(sequences_digit, f)